# Izpit: Analitika 2: Strojno učenje v Python-u

Rok oddaje: `TODO!`

Cilj: `doseči čim boljšo končno napoved.`

Za vsa vprašanja smo na voljo.

Lahko si pomagate z uporabo gradiv in internetom. Ne pozabite na uradno dokumentacijo.

Srečno!!

# Problem 2: Regresijski

Podane imate podatke o stanovanjskih območjih in njihove povprečne karakteristike.

Atributi:
* `Prihodek` (povprečen prihodek v hiši)
* `StartostHise` (povprečna starost hiše)
* `SteviloSob` (povprečno število sob v hiši)
* `SteviloSpalnic` (povprečno število spalnic v hiši)
* `VzorcnoSteviloPrebivalcev` (število prebivalcev iz kjer smo izračunali povprečja)
* `SteviloPrebivalcev` (povprečno število prebivalcev v hiši)
* `ZemljepisnaSirina` (povprečna zemljepisna širina hiše)
* `ZemljepisnaDolzina` (povrečna zemljepisna dolžina hiše)

Ciljni atribut:
* `VrednostHise` (povprečna vrednost hiše)

Vaša naloga je, da izdelate regresijski model, ki bo čim boljše napovedoval povprečno ceno hiše glede na podane atribute.

Za napovedovanje lahko uporabite kakršnekoli metode.

Pred samo napovedjo boste morali značilke urediti v obliko, ki bo omogočala napovedovanje cene.

```
POZOR: Pazite, da za optimizacijo modela uporabljate validacijske podatke, ki jih naredite z delitvijo train podatkov, že naloženih s spodnjo kodo. Spodnje celice ne spreminjajte, da vsi primerjamo rezultate z istimi testnimi podatki.
```

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("./data/Problem2_cena_hise.csv",sep="\t",encoding="utf-8")
train, test = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)
display(data)
display(train)
display(test)

#### Prvotna analiza podatkov

In [ ]:
display(data.shape)
display(data.info())
display(data.isnull().sum())
# vidimo da ni manjkajočih vrednosti -> imputacija ne bo potrebna


In [ ]:
display(data.describe())
# vsi podatki so že v numerični obliki -> ni potrebe po encodiranju podatkov

Dodatne ideje - todo!
- vizualiziraj outlinerje
- vizualiziraj medsebojne odvisnosti
- iz zemplepisne sirine in dolzine ustvari znacilko povprecne velikosti (footrprint)

In [ ]:
# Vizualizacija značilnosti podatkov, ki so pomembne za izbiro modela
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.PairGrid(data)
g.map(sns.scatterplot)
plt.show()

In [ ]:
sns.lmplot(x="StartostHise", y="VrednostHise", data=data, order=1)
plt.show()

In [ ]:
sns.lmplot(x="Prihodek", y="VrednostHise", data=data, order=1)
plt.show()

In [ ]:
sns.lmplot(x="ZemljepisnaSirina", y="ZemljepisnaDolzina", data=data, order=1)
plt.show()

#### Feature-engineering
- iz geo latitude & longitude s pomočjo reverse geocodinga pridobimo podatke o lokaciji

Spodnjo kodo uporabimo za reverse-geocoding. Ker izvajanje traja dolgo časa smo končni rezultat shranili v csv iz katerega beremo podatke v nadaljevanju.

``` python
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.point import Point

geolocator = Nominatim(user_agent="test")

def reverse_geocoding(lat, lon):
    try:
        location = geolocator.reverse(Point(lat, lon))
        return location.raw['display_name']
    except:
        return None

df = data
df['address'] = np.vectorize(reverse_geocoding)(df['ZemljepisnaSirina'], df['ZemljepisnaDolzina'])
df["Mesto"] = df["address"].str.split(',').str[-5]
df.drop(columns = ['Unnamed: 0'], inplace=True)
df.to_csv("./data/Problem2_cena_hise_reverse_geocoding.csv", sep="\t",encoding="utf-8", index=False)

```

- Prebermo podatke, kjer je stolpec Mesto že pripravljeno (Glede na kodo zgoraj)

In [ ]:
data_mesto = pd.read_csv("./data/Problem2_cena_hise_reverse_geocoding.csv",sep="\t",encoding="utf-8")
data_mesto.drop(columns = ['address'], inplace=True)
display(data_mesto.head(5))

#### Porazdelitev podatkov, distribucija značilk

In [ ]:
from scipy import stats
# Create a grid of subplots with matplotlib
fig, axes = plt.subplots(nrows=len(data.columns), ncols=1, figsize=(6, 30))

# Iterate over all columns and plot a probability plot on each subplot
for i, col in enumerate(data.columns):
    ax = axes[i]
    stats.probplot(data[col], dist="norm", plot=ax)
    ax.set_title(f"Probability Plot for {col}")

plt.tight_layout()
plt.show()

In [ ]:
data.hist(bins=30, figsize=(12,12), density=True)
plt.show()

#### Linear regresion

In [ ]:
# Train in test podatke razdelimo na znacilke in prediktorje za nadaljnjo uporabo
y_train = train['VrednostHise']
X_train = train.drop(columns=["VrednostHise"])
y_test = test['VrednostHise']
X_test = test.drop(columns=["VrednostHise"])

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
print(f"Training set score: {lr.score(X_train, y_train):.2f}")
print(f"Test set score: {lr.score(X_test, y_test):.2f}")

# Primer underfittinga -> preveč enostaven model, kar povzorči nizek accuracy na train in test podatkih.

- Dodamo skaliranje in regularizacijo

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # importanje L2 regularizacije

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
print(f"Training set score: {ridge.score(X_train_scaled, y_train):.2f}")
print(f"Test set score: {ridge.score(X_test_scaled, y_test):.2f}")